In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from model import *

unlabel_prob = 0.95 # probability of labels to be unlabeled
percentile = 0     # when a graph is not given, construct a graph 
                    # dropping nodes under this percentile 

sigma = 1.5           # sigma hyperparameter for RBF kernel

hard = False


%matplotlib inline

In [2]:
true_labels, features, graph = Data.Data.load_cora(multiclass=True)
labels, is_labeled, labeled_indices, unlabeled_indices = utils.random_unlabel(true_labels,unlabel_prob=unlabel_prob,multiclass=True)
solution  = true_labels[unlabeled_indices]
weights, graph   = utils.rbf_kernel(features,s=sigma,G=graph)

graph given


In [3]:
# baseline_prediction = np.rint(labels[unlabeled_indices])
# print("[Baseline]　Accuracy:", utils.accuracy(solution, baseline_prediction), 
#       "RMSE:", utils.rmse(solution, baseline_prediction))

In [8]:
lp = LP.LP()
lp_closed_prediction = utils.prob_to_one_hot(lp.closed(labels,
                                              weights,
                                              labeled_indices,
                                              unlabeled_indices))
lp_iter_prediction = utils.prob_to_one_hot(lp.iter(labels,
                                           weights,
                                           is_labeled,
                                           num_iter=100))

In [9]:
print("[Closed]　Accuracy:", utils.accuracy(solution, lp_closed_prediction,multiclass=True), 
      "RMSE:", utils.mse(solution, lp_closed_prediction,multiclass=True))
print("[Iter]　Accuracy:", utils.accuracy(solution, lp_iter_prediction[unlabeled_indices],multiclass=True), 
      "RMSE:", utils.mse(solution, lp_iter_prediction[unlabeled_indices],multiclass=True))

[Closed]　Accuracy: 0.761016949153 RMSE: 0.238983050847
[Iter]　Accuracy: 0.76186440678 RMSE: 0.23813559322
